In [208]:
import pandas as pd
import numpy as np
import os
import requests
import zipfile
import io
import glob
import re
from datetime import datetime

In [209]:
data_frame = pd.read_csv('CombinedData.csv')

/Users/vikasshah/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,19,49,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [210]:
df = data_frame

In [211]:
data_frame['member_id'].unique()

array([ nan])

In [212]:
## drop all those rows in which half the number of columns has null values in the data.
df= df.dropna(thresh=int(len(df.columns)/2))

In [213]:
df.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10119623,NaN,12000.0,12000.0,12000.0,36 months,11.99%,398.52,B,B3,...,0.0,3.0,100.0,1.0,0.0,0.0,365874.0,44327.0,10700.0,57674.0
1,10224583,NaN,11100.0,11100.0,11100.0,36 months,14.98%,384.68,C,C3,...,1.0,1.0,75.0,50.0,0.0,0.0,385000.0,6619.0,4000.0,0.0


In [7]:
### drop rows if it 

In [8]:
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns 

In [9]:
missing_values_table(df)

,Missing Values,% of Total Values
id,0,0.000000
member_id,1307181,100.000000
loan_amnt,0,0.000000
funded_amnt,0,0.000000
funded_amnt_inv,0,0.000000
term,0,0.000000
int_rate,0,0.000000
installment,0,0.000000
grade,0,0.000000
sub_grade,0,0.000000


In [214]:
## read first 10 columns to clean ..
df.ix[:1,0:10].head(2)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade
0,10119623,NaN,12000.0,12000.0,12000.0,36 months,11.99%,398.52,B,B3
1,10224583,NaN,11100.0,11100.0,11100.0,36 months,14.98%,384.68,C,C3


In [215]:
## remove months from column term and it has only 3 null values, replace it with ffill method.

df['term']=df['term'].str.split(' ',2).str[1]
df['term'] = df['term'].fillna(method='ffill')
df['term'] = df['term'].astype(int)
## remove % from column int_rate column and it has only 3 null values, replace it with ffill method.
df['int_rate'] = df['int_rate'].str.split('%',1).str[0]
df['int_rate'] =df['int_rate'].astype(float)

## 
df['loan_amnt'] = df['loan_amnt'].astype(int)
df['funded_amnt'] = df['funded_amnt'].astype(int)
df['installment'] = df['installment'].astype(int)

## grade and sub-grade has no null values.

/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:/

In [216]:
## read next 10 columns to clean ..
df.ix[:1,10:20].head(2)

,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc
0,LTC,10+ years,MORTGAGE,130000.0,Source Verified,Dec-2013,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN
1,Teacher,10+ years,MORTGAGE,90000.0,Not Verified,Dec-2013,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN


In [217]:
df.emp_length.unique()

array(['10+ years', '1 year', '2 years', '4 years', '7 years', '5 years',
       '6 years', '3 years', '8 years', '9 years', '< 1 year', 'n/a'], dtype=object)

In [218]:
## remove years and other character from emp_length to make it numerical 
df['emp_length_clean'] = df.emp_length.str.replace('+','')
df['emp_length_clean'] = df.emp_length_clean.str.replace('<','')
df['emp_length_clean'] = df.emp_length_clean.str.replace('years','')
df['emp_length_clean'] = df.emp_length_clean.str.replace('year','')
df['emp_length_clean'] = df.emp_length_clean.str.replace('n/a','0')
df['emp_length_clean']=pd.to_numeric(df['emp_length_clean'], errors='ignore')
df['emp_length_clean'].fillna(0,inplace=True)
df['emp_length_clean'].unique()
df['emp_length_clean'].astype(int)



/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

0          10
1          10
2           1
3          10
4           2
5           4
6          10
7          10
8           7
9          10
10          5
11          6
12         10
13          3
14          2
15         10
16          4
17          2
18         10
19          4
20         10
21          3
22          2
23         10
24          8
25          1
26          2
27         10
28         10
29         10
           ..
1319008     5
1319009     3
1319010     1
1319011     1
1319012     9
1319013    10
1319014     1
1319015     3
1319017     6
1319018     1
1319019     9
1319020     1
1319021     2
1319022     2
1319023    10
1319025     5
1319026     1
1319027     1
1319028     1
1319029     1
1319030     1
1319031     1
1319032     5
1319035     9
1319036    10
1319037     1
1319038     1
1319039     1
1319040     2
1319054     2
Name: emp_length_clean, dtype: int64

In [219]:
df.ix[:1,20:30].head(2)

,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths
0,debt_consolidation,Debt consolidation,809xx,CO,13.03,0.0,Nov-1997,715.0,719.0,1.0
1,other,Other,103xx,NY,3.73,1.0,Jun-2001,690.0,694.0,0.0


In [220]:
## dti, delinq_2yrs, fico_range_low, fico_range_high, earliest_cr_line has zero null values..
df.dti = df.dti.astype(int)
df.delinq_2yrs = df.delinq_2yrs.astype(int)
df.fico_range_low = df.fico_range_low.astype(int)
df.fico_range_high= df.fico_range_high.astype(int)
df.inq_last_6mths= df.inq_last_6mths.fillna(0)
df.inq_last_6mths = df.inq_last_6mths.astype(int)


/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [221]:
df.ix[:1,30:40].head(2)

,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv
0,NaN,NaN,9.0,0.0,10805.0,67%,19.0,f,0.0,0.0
1,16.0,NaN,9.0,0.0,6619.0,66.2%,12.0,f,0.0,0.0


In [222]:
## fill na
## mths_since_last_delinq has 646467 null values.. (do not think we need this for FE)
df.mths_since_last_delinq = df.mths_since_last_delinq.fillna(0).astype(int)

## mths_since_last_record has 1088168 null values.. (do not think we need this for FE)
df.mths_since_last_record = df.mths_since_last_record.fillna(0).astype(int)

##open_acc has 1088168 null values.. (do not think we need this for FE)
df.open_acc = df.open_acc.fillna(0).astype(int)

## pub_rec, revol_bal,total_acc has no null values
df.pub_rec = df.pub_rec.astype(int)
df.total_acc = df.total_acc.astype(int)
df.revol_bal = df.revol_bal.astype(int)
df.out_prncp = df.out_prncp.astype(int)
df.out_prncp_inv = df.out_prncp_inv.astype(int)

df['revol_util'] = df['revol_util'].str.split('%',1).str[0].astype(float)
## mean revol_util 
df['revol_util']=df['revol_util'].fillna((df.revol_util.mean())).astype(int)


/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/vikasshah/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [223]:
df.ix[:1,40:50].head(2)

,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d
0,14346.479050,14346.48,12000.0,2346.48,0.0,0.0,0.0,Jan-2017,398.28,NaN
1,13575.640006,13575.64,11100.0,2475.64,0.0,0.0,0.0,Feb-2016,4343.32,NaN


In [225]:
## total_pymnt,total_pymnt_inv
df.total_pymnt = df.total_pymnt.fillna(0).astype(float)
df.total_pymnt_inv = df.total_pymnt_inv.fillna(0).astype(float)
df.total_rec_prncp = df.total_rec_prncp.fillna(0).astype(float)
df.total_rec_int = df.total_rec_int.astype(float)
df.total_rec_late_fee = df.total_rec_late_fee.astype(float)
df.recoveries = df.recoveries.astype(float)
df.collection_recovery_fee = df.collection_recovery_fee.astype(float)
df.last_pymnt_d = df.last_pymnt_d.fillna(method='ffill')
df.last_pymnt_amnt = df.last_pymnt_amnt.fillna(0).astype(float)


/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [226]:
df.ix[:1,50:60].head(2)

,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint
0,Mar-2017,729.0,725.0,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN
1,Aug-2016,724.0,720.0,0.0,16.0,1.0,INDIVIDUAL,NaN,NaN,NaN


In [227]:
df.last_credit_pull_d = df.last_credit_pull_d.fillna(method='ffill')

df.last_fico_range_high = df.last_fico_range_high.astype(int)
df.last_fico_range_low = df.last_fico_range_low.astype(int)
df.collections_12_mths_ex_med = df.collections_12_mths_ex_med.fillna(0).astype(int)
df.mths_since_last_major_derog = df.mths_since_last_major_derog.fillna(0).astype(int)
df.policy_code = df.policy_code.astype(int)
df.application_type = df.application_type.fillna('unknown')
df.annual_inc_joint = df.annual_inc_joint.fillna(0)
df.dti_joint = df.dti_joint.fillna(0)
df.verification_status_joint = df.verification_status_joint.fillna(0)


/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [228]:
df.ix[:1,60:70].head(2)

,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util
0,0.0,0.0,327264.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,353402.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [231]:
df.acc_now_delinq = df.acc_now_delinq.astype(int)
df.tot_coll_amt = df.tot_coll_amt.fillna(0).astype(int)
df.tot_cur_bal = df.tot_cur_bal.fillna(0).astype(int)
df.open_acc_6m = df.open_acc_6m.fillna(0).astype(int)
df.open_il_6m = df.open_il_6m.fillna(0).astype(int)
df.open_il_12m =df.open_il_12m.fillna(0).astype(int)
df.open_il_24m = df.open_il_24m.fillna(0).astype(int)
df.mths_since_rcnt_il = df.mths_since_rcnt_il.fillna(0).astype(int)
df.total_bal_il =df.total_bal_il.fillna(0).astype(int)
df.il_util= df.il_util.fillna(0).astype(int)



/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [232]:
df.ix[:,70:80].head(2)

,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal
0,NaN,NaN,NaN,NaN,16200.0,NaN,NaN,NaN,4.0,36362.0
1,NaN,NaN,NaN,NaN,10000.0,NaN,NaN,NaN,2.0,39267.0


In [233]:
df.total_rev_hi_lim =df.total_rev_hi_lim.fillna(0).astype(float)
df.acc_open_past_24mths = df.acc_open_past_24mths.fillna(0).astype(int)
df.avg_cur_bal = df.avg_cur_bal.fillna(0).astype(float)
df.mths_since_recent_bc =df.mths_since_recent_bc.fillna(0).astype(int)

/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [234]:
df.ix[:,80:90].head(2)

,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc
0,3567.0,93.0,0.0,0.0,173.0,193.0,4.0,4.0,3.0,85
1,1016.0,74.6,0.0,0.0,NaN,150.0,11.0,11.0,1.0,11


In [236]:
df.bc_util =df.bc_util.fillna(0).astype(int)
df.delinq_amnt = df.delinq_amnt.fillna(0).astype(int)

/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [237]:
df.ix[:,90:100].head(2)

,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl
0,NaN,4.0,NaN,0.0,3.0,5.0,4.0,4.0,8.0,5.0
1,35.0,11.0,35.0,1.0,4.0,8.0,4.0,4.0,0.0,8.0


In [240]:
df.mths_since_recent_bc_dlq =df.mths_since_recent_bc_dlq.fillna(0).astype(int)
df.mths_since_recent_revol_delinq = df.mths_since_recent_revol_delinq.fillna(0)
df.tax_liens = df.tax_liens.astype(int)
df.tot_hi_cred_lim = df.tot_hi_cred_lim.fillna(0).astype(float)
df.total_il_high_credit_limit = df.total_il_high_credit_limit.fillna(0).astype(float)

/Users/vikasshah/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [241]:
del df['bc_open_to_buy']
del df['chargeoff_within_12_mths']
del df['mo_sin_old_il_acct']
del df['mo_sin_old_rev_tl_op']
del df['mo_sin_rcnt_rev_tl_op']
del df['mo_sin_rcnt_tl']
del df['mort_acc']
del df['mths_since_recent_bc']


del df['num_accts_ever_120_pd']
del df['num_actv_bc_tl']
del df['num_actv_rev_tl']
del df['num_bc_sats']
del df['num_bc_tl']
del df['num_il_tl']
del df['num_op_rev_tl']
del df['num_rev_accts']
del df['num_rev_tl_bal_gt_0']
del df['num_tl_120dpd_2m']
del df['num_tl_30dpd']
del df['num_tl_90g_dpd_24m']
del df['num_tl_op_past_12m']

del df['pct_tl_nvr_dlq']
del df['percent_bc_gt_75']
del df['pub_rec_bankruptcies']
del df['total_bal_ex_mort']

del df['total_bc_limit']
del df['total_bal_ex_mort']

del df['id']
del df['member_id']
del df['emp_title']
del df['desc']
del df['max_bal_bc']
del df['all_util']
del df['inq_fi']
del df['total_cu_tl']
del df['verification_status_joint']

In [263]:
##  delete columns

#del df['inq_fi']
#del df['total_cu_tl']
#del df['inq_last_12m']
#del df['total_bal_il']
#del df['il_util']
#del df['open_rv_12m']
#del df['mths_since_recent_revol_delinq']
#del df['mths_since_recent_bc_dlq']
#del df['open_acc_6m']
#del df['open_il_6m']
#del df['open_il_12m']
#del df['open_il_12m']
#del df['open_il_24m']
#del df['mths_since_rcnt_il']
#del df['desc','mths_since_last_record'] 
#del df['mths_since_last_major_derog']
#del df['annual_inc_joint','dti_joint']



In [264]:
df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state',
       'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low',
       'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high',
       'last_fico_range_low', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joi

In [266]:
#### continue with feature engineering....

In [267]:
#### continue with feature engineering....